### Explore Import Methods

In [30]:
import pandas as pd
import numpy as np

import os
import time

import modin.pandas as modin
from dask import dataframe as dd
import ray

In [31]:
os.path.getsize('/Users/devikachandnani/Library/Mobile Documents/com~apple~CloudDocs/Documents/Grad/Fall2022/Internship/Week 6/taxi_zone_lookup.csv')

12322

##### 1.Pandas

In [32]:
import pandas as pd
start = time.time()
df = pd.read_csv('/Users/devikachandnani/Library/Mobile Documents/com~apple~CloudDocs/Documents/Grad/Fall2022/Internship/Week 6/taxi_zone_lookup.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  0.0018541812896728516 sec


#### 2.Dask


In [33]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('/Users/devikachandnani/Library/Mobile Documents/com~apple~CloudDocs/Documents/Grad/Fall2022/Internship/Week 6/taxi_zone_lookup.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.005460977554321289 sec


#### 3.Modin/Ray

In [34]:
import modin.pandas as modin
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('/Users/devikachandnani/Library/Mobile Documents/com~apple~CloudDocs/Documents/Grad/Fall2022/Internship/Week 6/taxi_zone_lookup.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2022-10-11 18:42:17,505	INFO worker.py:1518 -- Started a local Ray instance.


Read csv with modin and ray:  0.0022089481353759766 sec


##### Certaintly Pandas has been fastest in the case of this dataset

### Create Utility File

In [48]:
%%writefile utilityfile.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing utilityfile.py


### Write YAML File

In [43]:
%%writefile file.yaml
file_type: csv
dataset_name: taxidata
file_name: taxi_zone_lookup
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - LocationID
    - Borough
    - Zone
    - service_zone

Overwriting file.yaml


### Validation Process

In [49]:
#reading configuration file
import utilityfile as util
config_data = util.read_config_file("file.yaml")

In [50]:
config_data

{'file_type': 'csv',
 'dataset_name': 'taxidata',
 'file_name': 'taxi_zone_lookup',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['LocationID', 'Borough', 'Zone', 'service_zone']}

In [54]:
#read using dynamic file configuration using YAML
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [53]:
#read using pandas
import pandas as pd
df_sample = pd.read_csv("taxi_zone_lookup.csv",delimiter=',')
df_sample.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [55]:
#test collumns
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['LocationID', 'Borough', 'Zone', 'service_zone'], dtype='object')
columns of YAML are: ['LocationID', 'Borough', 'Zone', 'service_zone']


In [56]:
if util.col_header_val(df,config_data)==0:
    print("Result: Validation FAILED")
else:
    print("Result: Collumn Calidation PASSED")
    

column name and column length validation passed
Result: Collumn Calidation PASSED


### Summary of the File

In [57]:
df.shape

(265, 4)

In [58]:
df.describe

<bound method NDFrame.describe of      locationid        borough                     zone service_zone
0             1            EWR           Newark Airport          EWR
1             2         Queens              Jamaica Bay    Boro Zone
2             3          Bronx  Allerton/Pelham Gardens    Boro Zone
3             4      Manhattan            Alphabet City  Yellow Zone
4             5  Staten Island            Arden Heights    Boro Zone
..          ...            ...                      ...          ...
260         261      Manhattan       World Trade Center  Yellow Zone
261         262      Manhattan           Yorkville East  Yellow Zone
262         263      Manhattan           Yorkville West  Yellow Zone
263         264        Unknown                       NV          NaN
264         265        Unknown                      NaN          NaN

[265 rows x 4 columns]>